## Scraping CNN

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [12]:
# Mengecek Link status Ok
url = "https://www.cnnindonesia.com/search/?query=ganjar&page=2"
response = requests.get(url)
response

<Response [200]>

In [13]:
jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'prabowo'

In [14]:
def scrape_links(page_number,keywords):
    try:
        
        url = f"https://www.cnnindonesia.com/search/?query={keywords}&page={page_number}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all('article', {"class": "flex-grow"})
        
        page_links = []
        for article in articles:
              link_element = article.find('a')
              if link_element is not None:
                        link = link_element.get('href')
                        if link is not None:
                                page_links.append(link)
        
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    except Exception as e:
        print(f"Error while scraping page {page_number}: {str(e)}")
        return []

In [15]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 24 links from page 1
Total Links: 24


In [16]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "mb-2 text-[28px] leading-9 text-cnn_black"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('span', {"class": "text-cnn_red"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "text-cnn_grey text-sm mb-4"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('a', {"dtr-sec": "subkanal"})
            if category_elements:
                category_text= category_elements.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail-text text-cnn_black text-sm grow min-w-0"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [17]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [18]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(5)

hasil scrapping 24


,title,keywords,author,category,date,content,nama_berita,link
0,"Singkirkan Prancis, Indonesia Lolos ke Semifin...",prabowo,CNN Indonesia,Raket,"Jumat, 29 Sep 2023 07:08 WIB",Indonesia mengalahkan Prancis 3-1 untuk merebu...,cnnindonesia,https://www.cnnindonesia.com/olahraga/20230929...
1,Spesifikasi Piramid Toba yang Dapat Restu Luhu...,prabowo,CNN Indonesia,Sains,"Jumat, 29 Sep 2023 06:49 WIB","Struktur unik di bukit di kawasan Danau Toba, ...",cnnindonesia,https://www.cnnindonesia.com/teknologi/2023092...
2,Bulan Depan Jalanan Bersih dari Pelat RF,prabowo,CNN Indonesia,Tren,"Kamis, 28 Sep 2023 20:24 WIB",Pelat nomor khusus berkode RF yang sudah dihen...,cnnindonesia,https://www.cnnindonesia.com/otomotif/20230927...
3,"Pemilik Mobil Wajib Punya Kunci Ring, Berikut ...",prabowo,CNN Indonesia,Tren,"Jumat, 29 Sep 2023 07:00 WIB",Pemilik kendaraan harus memiliki ring wrench a...,cnnindonesia,https://www.cnnindonesia.com/otomotif/20230926...
4,Ramos Horta Puji Jokowi dan RI sampai Peta Bar...,prabowo,CNN Indonesia,Asia Pasifik,"Jumat, 29 Sep 2023 06:38 WIB",Presiden Joko Widodo (Jokowi) mengapresiasi Pr...,cnnindonesia,https://www.cnnindonesia.com/internasional/202...
5,"PDIP Gelar Rakernas Hari Ini, Mega dan Jokowi ...",prabowo,CNN Indonesia,Politik,"Jumat, 29 Sep 2023 07:50 WIB",PDI Perjuangan (PDIP) menyelenggarakan Rapat K...,cnnindonesia,https://www.cnnindonesia.com/nasional/20230929...
6,BCA Akan Tutup Rekening Saldo 0 Mulai 1 November,prabowo,CNN Indonesia,Keuangan,"Jumat, 29 Sep 2023 07:10 WIB",PT Bank Central Asia Tbk. (BBCA) mengumumkan p...,cnnindonesia,https://www.cnnindonesia.com/ekonomi/202309290...
7,Link Live Streaming Indonesia vs China di Bere...,prabowo,CNN Indonesia,Raket,"Jumat, 29 Sep 2023 07:24 WIB",Indonesia akan menghadapi China pada babak per...,cnnindonesia,https://www.cnnindonesia.com/olahraga/20230929...
8,"Penembakan di Rotterdam, Dua Orang Dilaporkan ...",prabowo,CNN Indonesia,Eropa Amerika,"Jumat, 29 Sep 2023 03:30 WIB",Dua orang tewas dan satu orang lainnya kritis ...,cnnindonesia,https://www.cnnindonesia.com/internasional/202...
9,Epic Games Bakal PHK 830 Karyawan,prabowo,CNN Indonesia,Bisnis,"Jumat, 29 Sep 2023 07:50 WIB","Epic Games, pembuat Fortnite, mengumumkan akan...",cnnindonesia,https://www.cnnindonesia.com/ekonomi/202309290...


In [19]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
filtered_df.head(10)

,title,keywords,author,category,date,content,nama_berita,link
12,"Pilih Dukung Anies, Kader Golkar Medan Mengund...",prabowo,CNN Indonesia,Pemilu,"Jumat, 29 Sep 2023 07:25 WIB",Politisi Golkar yang juga menjabat sebagai ang...,cnnindonesia,https://www.cnnindonesia.com/nasional/20230929...
23,Tanya Prabowo ke Kaesang di Ultah Luhut: Kapan...,prabowo,CNN Indonesia,Politik,"Jumat, 29 Sep 2023 08:15 WIB",Menteri Pertahanan yang juga bakal calon presi...,cnnindonesia,https://www.cnnindonesia.com/nasional/20230929...


Save Data

In [20]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/cnn_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')